## Reserved Keywords:

- `_all_` : Objects assigned to a variable with name `_all_` will be added to the `__all__` variable in the exported file. Lists, Tuples, Sets, String, and direct assignments are allowed. Using Starred or Logical expressions, Dicts, and having more than one thing on the left side of the equals are not allowed however.
    - `_all_ = some_function`
    - `_all_ = 'some_function'`
    - `_all_ = [func_a, 'func_b', {(class_a), class_b}, (var_a, func_c)]`

## Reserved Comments:

- `+default_exp` : Set the default export path and name
    - `auto` : Figure out what to use based on the relative directory of the notebook and the notebook name

- `+export` : This cell will be exported to the .py file described by `+default_exp`, or to the one given as a parameter to this keyword

- `meta` : This cell is run at export-time, and its resulting state dict is used by the exporting process. A use case could be setting export parameters, or changing functions, or add aditional functionality to the export process.

## Reserved File Names:

- `inpex.ipynb` : Used to generate a README.md

- Any file starting with a underscore `_` is ignored

## Comment Syntax:

```python
# +command_name -arg1 value1 -arg2 value2 -bool_arg
```

Using a command comment is done by starting a comment at 0 indentation

## Compilation Steps:

### Until Now:

- Search for `default_exp` keyword
- iterate over all comments in all cells:
    - if comment starts with `# export`
        - parse any options passed along
        - remove the comment from the source
        - fix imports of own submodules by making them relative imports
        - put the cell source code and options in a list
    - if comment starts with `# hide`, ignore this cell
- return list of source code and options
- iterate over all cells to be exported
    - parse cell source code into a python abstract syntax tree
    - iterate over all 0-indented nodes
        - if variable assignment:
            - handle compiler reserved keywords (`_all_`)
            - add names of all variables assigned to to set of names
        - if function or class definition:
            - handle (e.g. fastai) custom python functionality (`@patch`, `@typedispatch` decorators)
            - add name to set of names
    - return set of names
    - add origin information to each cell source code
    - add source code to ExportCache
    - add set of names to ExportCache
- generate "autogenerated" warning for top of export file
- combine set of names to `__all__ = []` list
- combine warning, names and source code to one string
- create folders
- write file to disk

### In the future: (This is almost entirely inaccurate compared to what is actually happening now)

#### Default:

- for all cells [in parallel]:
    - iterate over all comments in cell:
        - if comment starts with `#hide`, ignore this cell
        - if comment starts with `#export`
            - parse any options passed along
            - remove the comment from the source
            - parse cell source code into a python abstract syntax tree
            - iterate over all 0-indented nodes:
                - if option `internal` is `False`:
                    - if variable assignment:
                        - handle compiler reserved keywords (`_all_`)
                        - add names of all variables assigned to to set of names
                    - if function or class definition:
                        - handle (e.g. fastai) custom python functionality (`@patch`, `@typedispatch` decorators)
                        - add name to set of names
                - if import or import from: (not quite correct, since imports can also happen inside e.g. a function)
                    - if importing from own submodules:
                        - modify source code, making them relative imports instead of absolute
                - return set of names
            - store the cell source code and options
        - return list of source code and options
 

- if a comment starts with any of the registered commands
- parse command arguments based on signature of function co-reginstered with the command
- __TODO__: try all of this out. idk what the right thing to do is here
- call function with those arguments and receive another function as return value
- call returned function with first arg: cell source code and second arg: tuple of compiler metadata (Flags, lineno, etc.)
- Based on return value:
    - Either: Keep searching for commands
    - OR: skip this cell __IDEA__: Use exceptions for control flow?
    - OR: store result and continue with next cell

#### Using meta programming: (These are only thoughts on how this _could_ work)

All `meta` cells are executet as code.  
They can be used to:
- set FLAGS
- add new commands
- add custom rules for detecting comments
- do things like type checking
- inject custom logic into the compiler by redefining/wrapping existing functions or defining new functions

They are executed in the order in which they are defined, and after each cell, the compilers own internal python interpreter state is updated with the `meta` cells state dict.  
__TODO__: Check that this actually works lol  
__NOTE__: Maybe just execute the code directly with `exec()`, using the compilers state dict.  
__NOTE__: Doing that will most likely lead to inconsistencies over different versions of this project. A better solution might be to have a standardised interface that is mapped to and from by the compiler.

Find a cell containing `default_exp` to know where to export code cells to by default.

Find comments corresponding to registered commands, decode the arguments, and save the result together with the cell source code in a list.